In [1]:
%%file setup.py
from distutils.core import setup
from Cython.Build import cythonize

setup(
  name = 'nbody_cython',
  ext_modules = cythonize("nbody_cython.pyx"),
)

Overwriting setup.py


In [18]:
%%file nbody_cython.pyx
 

from itertools import combinations


cdef float PI 
PI = 3.14159265358979323

cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
cdef float dt
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[0] -= dx * body1_velocity_arg
    v1[1] -= dy * body1_velocity_arg
    v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag            
    v2[0] += dx * body2_velocity_arg
    v2[1] += dy * body2_velocity_arg
    v2[2] += dz * body2_velocity_arg

def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
def advance(dt=dt, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        advance the system one timestep
    '''
    
    list(map(to_do, BODIES1, BODIES2))      
    list(map(update_rs_for_body, BODIES_KEYS))
    
    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    print("HI3")
    for _ in range(loops):
        for _ in range(iterations):            
            for b1, b2 in zip(BODIES1, BODIES2):
                to_do(b1, b2)     
            for b in BODIES_KEYS:
                update_rs_for_body(b)
        print(report_energy())
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)


Overwriting nbody_cython.pyx


In [19]:
#!rm nbody_cython.*

In [20]:
!python setup.py build_ext --inplace

Compiling nbody_cython.pyx because it changed.
Cythonizing nbody_cython.pyx
running build_ext
building 'nbody_cython' extension
/usr/bin/clang -Wno-unused-result -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -I/Users/akunas/anaconda/include -arch x86_64 -I/Users/akunas/anaconda/include/python3.4m -c nbody_cython.c -o build/temp.macosx-10.6-x86_64-3.4/nbody_cython.o
nbody_cython.c:6526:28: warning: unused function '__Pyx_PyObject_AsString'
      [-Wunused-function]
static CYTHON_INLINE char* __Pyx_PyObject_AsString(PyObject* o) {
                           ^
nbody_cython.c:6523:32: warning: unused function '__Pyx_PyUnicode_FromString'
      [-Wunused-function]
static CYTHON_INLINE PyObject* __Pyx_PyUnicode_FromString(const char* c_str) {
                               ^
nbody_cython.c:6638:33: warning: unused function '__Pyx_PyIndex_AsSsize_t'
      [-Wunused-function]
static CYTHON_INLINE Py_ssize_t __Pyx_PyIndex_AsSsize_t(PyObject* b) {
                                ^
nbody_cyth

In [21]:
%reload_ext Cython
import pyximport
pyximport.install(reload_support=True)

(None, None)

In [22]:
import nbody_cython

In [25]:
pyximport.

<module 'pyximport.pyximport' from '/Users/akunas/anaconda/envs/py35/lib/python3.5/site-packages/pyximport/pyximport.py'>

In [23]:
import importlib
nbody_cython = importlib.reload(nbody_cython)

In [24]:
nbody_cython.nbody(100, 'sun', 20000)

HI2
-0.16924923339854453
-0.16926022362920234
-0.16925260111702572
-0.16923373679589856
-0.16921141887400096
-0.16919044953162257
-0.16917121485513795
-0.16916589047561525
-0.16917897959979222
-0.1692106405409532
-0.16924487527676485
-0.16926076196731749
-0.16925525947417136
-0.16923914894345843
-0.16921656144792865
-0.1691939243267383
-0.16917553401794466
-0.1691645157769394
-0.16917350430846342
-0.16920278375338976


KeyboardInterrupt: 

In [93]:
%%cython
 

from itertools import combinations


cdef float PI 
PI = 3.14159265358979323

cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24

BODIES = {
    'sun': ([0.0, 0.0, 0.0], [0.0, 0.0, 0.0], SOLAR_MASS),

    'jupiter': ([4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
                [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
                9.54791938424326609e-04 * SOLAR_MASS),

    'saturn': ([8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
               [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
               2.85885980666130812e-04 * SOLAR_MASS),

    'uranus': ([1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
               [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
               4.36624404335156298e-05 * SOLAR_MASS),

    'neptune': ([1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
                [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],
                5.15138902046611451e-05 * SOLAR_MASS)}

VISIT_SCHEDULE = list(combinations(BODIES.keys(), 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = list(BODIES.keys())
cdef float dt
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    ([x1, y1, z1], v1, m1) = BODIES[body1]
    ([x2, y2, z2], v2, m2) = BODIES[body2]
    (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    v1[0] -= dx * body1_velocity_arg
    v1[1] -= dy * body1_velocity_arg
    v1[2] -= dz * body1_velocity_arg

    body2_velocity_arg = m1 * mag            
    v2[0] += dx * body2_velocity_arg
    v2[1] += dy * body2_velocity_arg
    v2[2] += dz * body2_velocity_arg

def update_rs_for_body(body, BODIES=BODIES, dt=dt):
    (r, [vx, vy, vz], m) = BODIES[body]
    r[0] += dt * vx
    r[1] += dt * vy
    r[2] += dt * vz 
    
def advance(dt=dt, VISIT_SCHEDULE=VISIT_SCHEDULE, BODIES=BODIES, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        advance the system one timestep
    '''
    
    list(map(to_do, BODIES1, BODIES2))      
    list(map(update_rs_for_body, BODIES_KEYS))
    
    
def report_energy(e=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        compute the energy and return it so that it can be printed
    '''
    for (body1, body2) in VISIT_SCHEDULE:
        ((x1, y1, z1), v1, m1) = BODIES[body1]
        ((x2, y2, z2), v2, m2) = BODIES[body2]

        (dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        e += m * (vx * vx + vy * vy + vz * vz) / 2.
        
    return e

def offset_momentum(ref, px=0.0, py=0.0, pz=0.0, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    for body in BODIES_KEYS:
        (r, [vx, vy, vz], m) = BODIES[body]
        px -= vx * m
        py -= vy * m
        pz -= vz * m
        
    (r, v, m) = ref
    v[0] = px / m
    v[1] = py / m
    v[2] = pz / m


def nbody(loops, reference, iterations, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(BODIES[reference])

    print("HI3")
    for _ in range(loops):
        for _ in range(iterations):            
            for b1, b2 in zip(BODIES1, BODIES2):
                to_do(b1, b2)     
            for b in BODIES_KEYS:
                update_rs_for_body(b)
        print(report_energy())
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)


In [28]:

MASSES = np.array([
SOLAR_MASS,
9.54791938424326609e-04 * SOLAR_MASS,
2.85885980666130812e-04 * SOLAR_MASS,
4.36624404335156298e-05 * SOLAR_MASS,
5.15138902046611451e-05 * SOLAR_MASS,
])

NameError: name 'np' is not defined

In [27]:
nbody(100, 'sun', 20000)

HI3
-0.1690893079068559
-0.16905245121468515
-0.16901479020554386
-0.16902311933067754
-0.1690799004285987
-0.16908383254327716
-0.1690461736204416
-0.16901421104130873
-0.16902782318037898
-0.1690837516129988
-0.1690808313007166
-0.16904479509465253
-0.16901201255756795
-0.1690332033017434
-0.1690878781527322
-0.16908025201132323
-0.16904032859383303
-0.16900671492886093
-0.16903862183789334
-0.1690928210860594
-0.16907715613822444
-0.16903187697643296


KeyboardInterrupt: 

In [61]:
#%%cython
 

from itertools import combinations
import numpy as np

#cdef float PI 
PI = 3.14159265358979323

#cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

#cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24


BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
BODIES_INDICES = np.array(list(range(len(BODIES_NAMES))))
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
    [1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
    [1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
])

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
    
     [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
])

MASSES = np.array([
SOLAR_MASS,
9.54791938424326609e-04 * SOLAR_MASS,
2.85885980666130812e-04 * SOLAR_MASS,
4.36624404335156298e-05 * SOLAR_MASS,
5.15138902046611451e-05 * SOLAR_MASS,
])


VISIT_SCHEDULE = list(combinations(BODIES_INDICES, 2))
BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
BODIES_KEYS = BODIES_INDICES
KEYS = BODIES_INDICES
#cdef float dt
dt = 0.01

def to_do(body1, body2, BODIES=BODIES, dt=dt):
    xyz1 = POSITIONS[body1]
    xyz2 = POSITIONS[body2]
    
    v1 = VELOCITIES[body1]
    v2 = VELOCITIES[body2]
    
    m1 = MASSES[body1]
    m2 = MASSES[body2]
    
    #([x1, y1, z1], v1, m1) = BODIES[body1]
    #([x2, y2, z2], v2, m2) = BODIES[body2]
    
    
    d = xyz1 - xyz2
    #(dx, dy, dz) = (x1-x2, y1-y2, z1-z2)
    
    mag = dt * np.inner(d, d) ** (-1.5)
    #mag = dt * ((dx * dx + dy * dy + dz * dz) ** (-1.5))

    body1_velocity_arg = m2 * mag
    VELOCITIES[body1] -= d * body1_velocity_arg
    
    #v1[0] -= dx * body1_velocity_arg
    #v1[1] -= dy * body1_velocity_arg
    #v1[2] -= dz * body1_velocity_arg


    body2_velocity_arg = m1 * mag   
    VELOCITIES[body2] -= d * body2_velocity_arg

    #v2[0] += dx * body2_velocity_arg
    #v2[1] += dy * body2_velocity_arg
    #v2[2] += dz * body2_velocity_arg

    
def to_do_simplified(body1, body2, POSITIONS, VELOCITIES, MASSES):
    
    xyz1 = POSITIONS[body1]
    xyz2 = POSITIONS[body2]

    v1 = VELOCITIES[body1]
    v2 = VELOCITIES[body2]

    m1 = MASSES[body1]
    m2 = MASSES[body2]
                
    d = xyz1 - xyz2
    
    mag = dt * (np.inner(d, d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    VELOCITIES[body1] -= d * body1_velocity_arg

    body2_velocity_arg = m1 * mag   
    VELOCITIES[body2] += d * body2_velocity_arg
    

    
def update_rs_for_body(body, POSITIONS, VELOCITIES, MASSES, dt):
    #r = POSITIONS[body]
    #v = VELOCITIES[body]
    #m = MASSES[body]    
    
    #r += dt * v
    POSITIONS[body] += dt * VELOCITIES[body]
    #(r, [vx, vy, vz], m) = BODIES[body]
    #r[0] += dt * vx
    #r[1] += dt * vy
    #r[2] += dt * vz 
    
    
def report_energy(POSITIONS, VELOCITIES, MASSES, KEYS, VISIT_SCHEDULE):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = 0.0
    for (body1, body2) in VISIT_SCHEDULE:
        xyz1 = POSITIONS[body1]
        xyz2 = POSITIONS[body2]

        v1 = VELOCITIES[body1]
        v2 = VELOCITIES[body2]

        m1 = MASSES[body1]
        m2 = MASSES[body2]

        d = xyz1 - xyz2
        e -= (m1 * m2) / np.sqrt(np.inner(d, d))
        #(dx, dy, dz) = (x1-x2, y1-y2, z1-z2)

        #e -= (m1 * m2) / ((dx * dx + dy * dy + dz * dz) ** 0.5)
        
    for body in KEYS:
        r = POSITIONS[body]
        v = VELOCITIES[body]
        m = MASSES[body]   
        #(r, [vx, vy, vz], m) = BODIES[body]
        #e += m * (vx * vx + vy * vy + vz * vz) / 2.
        e += m * np.inner(v, v) / 2.0
    return e




def offset_momentum(reference_index, POSITIONS, VELOCITIES, MASSES, KEYS):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.array([0.0, 0.0, 0.0])
    for body in KEYS:
        r = POSITIONS[body]
        v = VELOCITIES[body]
        m = MASSES[body]
        #(r, [vx, vy, vz], m) = BODIES[body]
        #px -= vx * m
        #py -= vy * m
        #pz -= vz * m
        p -= v * m

    r = POSITIONS[reference_index]
    v = VELOCITIES[reference_index]
    m = MASSES[reference_index]    
    
    VELOCITIES[reference_index] = p / m



def nbody(loops, reference, iterations): #, BODIES=BODIES, BODIES_KEYS=BODIES_KEYS, BODIES1=BODIES1, BODIES2=BODIES2):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(0, POSITIONS, VELOCITIES, MASSES, KEYS)

    print("HI3")
    for _ in range(loops):
        for _ in range(iterations):            
            for body1, body2 in VISIT_SCHEDULE:
                to_do_simplified(body1, body2, POSITIONS, VELOCITIES, MASSES)   
            for b in BODIES_KEYS:
                update_rs_for_body(b, POSITIONS, VELOCITIES, MASSES, dt)
        print(report_energy(POSITIONS, VELOCITIES, MASSES, KEYS, VISIT_SCHEDULE))
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)


HI3
-0.169089262755
-0.169052404924


KeyboardInterrupt: 

In [ ]:
A = np.array([1,2,3])
def f(A):
    A[2] += 4
    return A

f(A)

In [52]:
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
    [1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
    [1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
])

In [54]:
POSITIONS[0]

array([ 0.,  0.,  0.])

In [57]:
np.inner([1,2,3], [1,2,3])

14

In [59]:
1**2 + 2**2 + 3**2

14

In [97]:
%%cython
 

from itertools import combinations
import numpy as np
cimport numpy as np

cdef float PI 
PI = 3.14159265358979323

cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24

ctypedef np.float_t DTYPE_FLOAT_t
ctypedef np.int_t DTYPE_INT_t

BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
KEYS = np.array(list(range(len(BODIES_NAMES))))
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
    [1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
    [1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
])

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
    
     [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
])

MASSES = np.array([
SOLAR_MASS,
9.54791938424326609e-04 * SOLAR_MASS,
2.85885980666130812e-04 * SOLAR_MASS,
4.36624404335156298e-05 * SOLAR_MASS,
5.15138902046611451e-05 * SOLAR_MASS,
])


VISIT_SCHEDULE = list(combinations(KEYS, 2))
VISIT_SCHEDULE = np.array(VISIT_SCHEDULE)
#BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
#BODIES_KEYS = BODIES_INDICES
#KEYS = BODIES_INDICES
cdef float dt
dt = 0.01

    
def to_do_simplified(int body1,
                     int body2,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                    np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                    ):
    
    xyz1 = POSITIONS[body1]
    xyz2 = POSITIONS[body2]

    v1 = VELOCITIES[body1]
    v2 = VELOCITIES[body2]

    m1 = MASSES[body1]
    m2 = MASSES[body2]
                
    d = xyz1 - xyz2
    
    mag = dt * (np.inner(d, d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    VELOCITIES[body1] -= d * body1_velocity_arg

    body2_velocity_arg = m1 * mag   
    VELOCITIES[body2] += d * body2_velocity_arg
    

    
def update_rs_for_body(int body,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                       np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                       float dt):
    POSITIONS[body] += dt * VELOCITIES[body]

    
    
def report_energy( np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS,
                   np.ndarray[DTYPE_INT_t, ndim=2] VISIT_SCHEDULE):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = 0.0
    for (body1, body2) in VISIT_SCHEDULE:
        xyz1 = POSITIONS[body1]
        xyz2 = POSITIONS[body2]

        v1 = VELOCITIES[body1]
        v2 = VELOCITIES[body2]

        m1 = MASSES[body1]
        m2 = MASSES[body2]

        d = xyz1 - xyz2
        e -= (m1 * m2) / np.sqrt(np.inner(d, d))

        
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]   

        e += m * np.inner(v, v) / 2.0
    return e




def offset_momentum(int reference_index,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS
                   ):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.array([0.0, 0.0, 0.0])
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]

        p -= v * m

    m = MASSES[reference_index]    
    
    VELOCITIES[reference_index] = p / m



def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(0, POSITIONS, VELOCITIES, MASSES, KEYS)

    print("HI3")
    for _ in range(loops):
        for _ in range(iterations):            
            for body1, body2 in VISIT_SCHEDULE:
                to_do_simplified(body1, body2, POSITIONS, VELOCITIES, MASSES)   
            for b in KEYS:
                update_rs_for_body(b, POSITIONS, VELOCITIES, MASSES, dt)
        print(report_energy(POSITIONS, VELOCITIES, MASSES, KEYS, VISIT_SCHEDULE))
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)

nbody(100, 'sun', 20000)


HI3
-0.169089307907
-0.169052451215
-0.169014790206
-0.169023119331
-0.169079900429
-0.169083832543
-0.16904617362
-0.169014211041
-0.16902782318
-0.169083751613
-0.169080831301
-0.169044795095
-0.169012012558
-0.169033203302
-0.169087878153
-0.169080252011
-0.169040328594
-0.169006714929
-0.169038621838


KeyboardInterrupt: 

In [98]:
%%cython
"""
A working version

"""

from itertools import combinations
import numpy as np
cimport numpy as np

cdef float PI 
PI = 3.14159265358979323

cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24

ctypedef np.float_t DTYPE_FLOAT_t
ctypedef np.int_t DTYPE_INT_t

BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
KEYS = np.array(list(range(len(BODIES_NAMES))))
POSITIONS = np.array([
    [0.0, 0.0, 0.0],
    [4.84143144246472090e+00,
                 -1.16032004402742839e+00,
                 -1.03622044471123109e-01],
    [8.34336671824457987e+00,
                4.12479856412430479e+00,
                -4.03523417114321381e-01],
    [1.28943695621391310e+01,
                -1.51111514016986312e+01,
                -2.23307578892655734e-01],
    [1.53796971148509165e+01,
                 -2.59193146099879641e+01,
                 1.79258772950371181e-01],
])

VELOCITIES = np.array([
    [0.0, 0.0, 0.0],
    [1.66007664274403694e-03 * DAYS_PER_YEAR,
                 7.69901118419740425e-03 * DAYS_PER_YEAR,
                 -6.90460016972063023e-05 * DAYS_PER_YEAR],
    [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                4.99852801234917238e-03 * DAYS_PER_YEAR,
                2.30417297573763929e-05 * DAYS_PER_YEAR],
    
     [2.96460137564761618e-03 * DAYS_PER_YEAR,
                2.37847173959480950e-03 * DAYS_PER_YEAR,
                -2.96589568540237556e-05 * DAYS_PER_YEAR],
    [2.68067772490389322e-03 * DAYS_PER_YEAR,
                 1.62824170038242295e-03 * DAYS_PER_YEAR,
                 -9.51592254519715870e-05 * DAYS_PER_YEAR],  
])

MASSES = np.array([
SOLAR_MASS,
9.54791938424326609e-04 * SOLAR_MASS,
2.85885980666130812e-04 * SOLAR_MASS,
4.36624404335156298e-05 * SOLAR_MASS,
5.15138902046611451e-05 * SOLAR_MASS,
])


VISIT_SCHEDULE = list(combinations(KEYS, 2))
VISIT_SCHEDULE = np.array(VISIT_SCHEDULE)
#BODIES1, BODIES2= zip(*VISIT_SCHEDULE)
#BODIES_KEYS = BODIES_INDICES
#KEYS = BODIES_INDICES
cdef float dt
dt = 0.01

    
def to_do_simplified(int body1,
                     int body2,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                    np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                    ):
    
    xyz1 = POSITIONS[body1]
    xyz2 = POSITIONS[body2]

    v1 = VELOCITIES[body1]
    v2 = VELOCITIES[body2]

    m1 = MASSES[body1]
    m2 = MASSES[body2]
                
    d = xyz1 - xyz2
    
    mag = dt * (np.inner(d, d) ** (-1.5))

    body1_velocity_arg = m2 * mag
    VELOCITIES[body1] -= d * body1_velocity_arg

    body2_velocity_arg = m1 * mag   
    VELOCITIES[body2] += d * body2_velocity_arg
    

    
def update_rs_for_body(int body,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                       np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                       float dt):
    POSITIONS[body] += dt * VELOCITIES[body]

    
    
def report_energy( np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS,
                   np.ndarray[DTYPE_INT_t, ndim=2] VISIT_SCHEDULE):
    '''
        compute the energy and return it so that it can be printed
    '''
    e = 0.0
    for (body1, body2) in VISIT_SCHEDULE:
        xyz1 = POSITIONS[body1]
        xyz2 = POSITIONS[body2]

        v1 = VELOCITIES[body1]
        v2 = VELOCITIES[body2]

        m1 = MASSES[body1]
        m2 = MASSES[body2]

        d = xyz1 - xyz2
        e -= (m1 * m2) / np.sqrt(np.inner(d, d))

        
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]   

        e += m * np.inner(v, v) / 2.0
    return e




def offset_momentum(int reference_index,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS
                   ):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    p = np.array([0.0, 0.0, 0.0])
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]

        p -= v * m

    m = MASSES[reference_index]    
    
    VELOCITIES[reference_index] = p / m



def nbody(loops, reference, iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # Set up global state
    offset_momentum(0, POSITIONS, VELOCITIES, MASSES, KEYS)

    print("HI3")
    for _ in range(loops):
        for _ in range(iterations):            
            for body1, body2 in VISIT_SCHEDULE:
                to_do_simplified(body1, body2, POSITIONS, VELOCITIES, MASSES)   
            for b in KEYS:
                update_rs_for_body(b, POSITIONS, VELOCITIES, MASSES, dt)
        print(report_energy(POSITIONS, VELOCITIES, MASSES, KEYS, VISIT_SCHEDULE))
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)

nbody(100, 'sun', 20000)


HI3
-0.169089307907
-0.169052451215
-0.169014790206


KeyboardInterrupt: 

In [82]:
%%cython
import numpy as np
cimport numpy as np

A = np.array([[1.0, 2.0, 3.0, 4.0], [1.0, 2.0, 3.0, 4.0]])

ctypedef np.float_t DTYPE_t

def change_second_value(np.ndarray[np.float_t, ndim=2] A):
#    A[1, 1] = 999.0
    a = A[0]
    a[1] = 999.0
    

change_second_value(A)
print(A)

[[   1.  999.    3.    4.]
 [   1.    2.    3.    4.]]


In [91]:
KEYS = [1,2,3]
VISIT_SCHEDULE = list(combinations(KEYS, 2))
VISIT_SCHEDULE = np.array(VISIT_SCHEDULE)

for a, b in VISIT_SCHEDULE:
    print(a, b)

1 2
1 3
2 3


# Trying to make it better

In [119]:
%%file nbody_cython.py
#%%cython

"""
A working version

"""

from itertools import combinations
import numpy as np
cimport numpy as np
cimport cython

ctypedef np.float_t DTYPE_FLOAT_t
ctypedef np.int_t DTYPE_INT_t

@cython.boundscheck(False)
cdef void to_do_simplified(int body1,
                     int body2,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                    np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                    np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                    float dt
                    ):
    
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] xyz1 = POSITIONS[body1]
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] xyz2 = POSITIONS[body2]

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] v1 = VELOCITIES[body1]
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] v2 = VELOCITIES[body2]

    cdef float m1 = MASSES[body1]
    cdef float m2 = MASSES[body2]
                
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] d = xyz1 - xyz2
    
    cdef float mag = dt * (np.inner(d, d) ** (-1.5))

    cdef body1_velocity_arg = m2 * mag
    VELOCITIES[body1] -= d * body1_velocity_arg

    cdef body2_velocity_arg = m1 * mag   
    VELOCITIES[body2] += d * body2_velocity_arg
    

@cython.boundscheck(False)    
cdef void update_rs_for_body(int body,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                       np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                       np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                       float dt):
    POSITIONS[body] += dt * VELOCITIES[body]

    
@cython.boundscheck(False)    
cdef float report_energy( np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS,
                   np.ndarray[DTYPE_INT_t, ndim=2] VISIT_SCHEDULE):
    '''
        compute the energy and return it so that it can be printed
    '''
    cdef float e = 0.0
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] xyz1, xyz2, d
    cdef float m1, m2
    for (body1, body2) in VISIT_SCHEDULE:
        xyz1 = POSITIONS[body1]
        xyz2 = POSITIONS[body2]


        m1 = MASSES[body1]
        m2 = MASSES[body2]

        d = xyz1 - xyz2
        e -= (m1 * m2) / np.sqrt(np.inner(d, d))

        
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]   

        e += m * np.inner(v, v) / 2.0
    return e



@cython.boundscheck(False)
cdef void offset_momentum(int reference_index,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS,
                   np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES,
                   np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES,
                   np.ndarray[DTYPE_INT_t, ndim=1] KEYS
                   ):
    '''
        ref is the body in the center of the system
        offset values from this reference
    '''
    cdef np.ndarray p = np.array([0.0, 0.0, 0.0])
    for body in KEYS:
        v = VELOCITIES[body]
        m = MASSES[body]

        p -= v * m

    m = MASSES[reference_index]    
    
    VELOCITIES[reference_index] = p / m



cdef nbody(int loops, str reference, int iterations):
    '''
        nbody simulation
        loops - number of loops to run
        reference - body at center of system
        iterations - number of timesteps to advance
    '''
    # initialize stuff
    cdef float PI 
    PI = 3.14159265358979323

    cdef float SOLAR_MASS
    SOLAR_MASS = 4 * PI * PI

    cdef float DAYS_PER_YEAR
    DAYS_PER_YEAR = 365.24



    BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
    cdef np.ndarray[DTYPE_INT_t, ndim=1] KEYS = np.arange(5)
    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] POSITIONS = np.array([
        [0.0, 0.0, 0.0],
        [4.84143144246472090e+00,
                     -1.16032004402742839e+00,
                     -1.03622044471123109e-01],
        [8.34336671824457987e+00,
                    4.12479856412430479e+00,
                    -4.03523417114321381e-01],
        [1.28943695621391310e+01,
                    -1.51111514016986312e+01,
                    -2.23307578892655734e-01],
        [1.53796971148509165e+01,
                     -2.59193146099879641e+01,
                     1.79258772950371181e-01],
    ])

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=2] VELOCITIES = np.array([
        [0.0, 0.0, 0.0],
        [1.66007664274403694e-03 * DAYS_PER_YEAR,
                     7.69901118419740425e-03 * DAYS_PER_YEAR,
                     -6.90460016972063023e-05 * DAYS_PER_YEAR],
        [-2.76742510726862411e-03 * DAYS_PER_YEAR,
                    4.99852801234917238e-03 * DAYS_PER_YEAR,
                    2.30417297573763929e-05 * DAYS_PER_YEAR],

         [2.96460137564761618e-03 * DAYS_PER_YEAR,
                    2.37847173959480950e-03 * DAYS_PER_YEAR,
                    -2.96589568540237556e-05 * DAYS_PER_YEAR],
        [2.68067772490389322e-03 * DAYS_PER_YEAR,
                     1.62824170038242295e-03 * DAYS_PER_YEAR,
                     -9.51592254519715870e-05 * DAYS_PER_YEAR],  
    ])

    cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] MASSES = np.array([
    SOLAR_MASS,
    9.54791938424326609e-04 * SOLAR_MASS,
    2.85885980666130812e-04 * SOLAR_MASS,
    4.36624404335156298e-05 * SOLAR_MASS,
    5.15138902046611451e-05 * SOLAR_MASS,
    ])


    cdef np.ndarray[DTYPE_INT_t, ndim=2] VISIT_SCHEDULE = np.array(list(combinations(KEYS, 2)))

    cdef float dt = 0.01
    # ================
    
    
    
    
    # Set up global state
    offset_momentum(BODIES_NAMES.index(reference), POSITIONS, VELOCITIES, MASSES, KEYS)

    for _ in range(loops):
        for _ in range(iterations):            
            for body1, body2 in VISIT_SCHEDULE:
                to_do_simplified(body1, body2, POSITIONS, VELOCITIES, MASSES, dt)   
            for b in KEYS:
                update_rs_for_body(b, POSITIONS, VELOCITIES, MASSES, dt)
        print(report_energy(POSITIONS, VELOCITIES, MASSES, KEYS, VISIT_SCHEDULE))
            

if __name__ == '__main__':
    nbody(100, 'sun', 20000)

nbody(100, 'sun', 20000)


Overwriting nbody_cython.py


In [103]:
%%cython

from itertools import combinations
import numpy as np
cimport numpy as np

cdef float PI 
PI = 3.14159265358979323

cdef float SOLAR_MASS
SOLAR_MASS = 4 * PI * PI

cdef float DAYS_PER_YEAR
DAYS_PER_YEAR = 365.24

ctypedef np.float_t DTYPE_FLOAT_t
ctypedef np.int_t DTYPE_INT_t

BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] KEYS = np.array(list(range(len(BODIES_NAMES))))


Error compiling Cython file:
------------------------------------------------------------
...

ctypedef np.float_t DTYPE_FLOAT_t
ctypedef np.int_t DTYPE_INT_t

BODIES_NAMES = ['sun', 'jupiter', 'saturn', 'uranus', 'neptune']
cdef np.ndarray[DTYPE_FLOAT_t, ndim=1] KEYS = np.array(list(range(len(BODIES_NAMES))))
                                      ^
------------------------------------------------------------

/Users/akunas/.ipython/cython/_cython_magic_aaaef87371a2f34157737cca6f79c235.pyx:19:39: Buffer types only allowed as function local variables

Error compiling Cython file:
------------------------------------------------------------
...

from itertools import combinations
^
------------------------------------------------------------

/Users/akunas/.ipython/cython/_cython_magic_aaaef87371a2f34157737cca6f79c235.pyx:2:0: Buffer vars not allowed in module scope


## Trying again